In [ ]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from keras.optimizers import adam, RMSprop
from keras.models import load_model
from data_preparation import StockIndexDataset
from rnn_models import *
from utils_rnn import plot_seqs, create_labels

In [ ]:
# Set constant variables and hyper-parameters for the model training

INPUT_PATH = "/kaggle/input/sh300if"  # OUTPUT_PATH = "/kaggle/output"  

RNN_params = {"batch_size": 32,
              "epochs": 120,
              "lr": 1e-4,
              "time_steps": 12,
              "forecast_steps": 6,
              "window": 12,
              "RNN_size": 128,
              "FC_size": 32,
              "optim": adam,
              "dropout": 0.5,
              "earlystop": {"patience": 20, 
                            "min_delta": 0.00001},
              "reduce_lr": {"factor": 0.1,
                            "patience": 10,
                            "min_delta": 0.0001}
              }

GRU_params = {"batch_size": 32,
              "epochs": 120,
              "lr": 1e-4,
              "time_steps": 6,
              "forecast_steps": 6,
              "window": 12,
              "RNN_size": 128,
              "FC_size": 32,
              "optim": adam,
              "dropout": 0.5,
              "earlystop": {"patience": 20, 
                            "min_delta": 0.00001},
              "reduce_lr": {"factor": 0.1,
                            "patience": 10,
                            "min_delta": 0.0001}
              }

LSTM_params = {"batch_size": 32,
               "epochs": 120,
               "lr": 1e-4,
               "time_steps": 6,
               "forecast_steps": 6,
               "window": 12,
               "RNN_size": 128,
               "FC_size": 32,
               "optim": adam,
               "dropout": 0.5,
               "earlystop": {"patience": 20, 
                             "min_delta": 0.00001},
               "reduce_lr": {"factor": 0.1,
                             "patience": 10,
                             "min_delta": 0.0001}
               }

In [ ]:
# Load datasets
datasets_path = [os.path.join(INPUT_PATH, file) for file in sorted(os.listdir(INPUT_PATH))]
for p in datasets_path:
    print(p)

In [ ]:
# Create a StockIndexDataset object for the SimpleRNN network
RNNdata = StockIndexDataset(datasets_path[0], time_steps=RNN_params['time_steps'], 
                            forecast_steps=RNN_params['forecast_steps'],
                            batch_size=RNN_params['batch_size'], val_ratio=.1, test_ratio=0.1)
RNNdata.info()

In [ ]:
# Create a StockIndexDataset object for the GRU network
GRUdata = StockIndexDataset(datasets_path[0], time_steps=GRU_params['time_steps'],
                            forecast_steps=GRU_params['forecast_steps'],
                            batch_size=GRU_params['batch_size'], val_ratio=.1, test_ratio=.1)
GRUdata.info()

In [ ]:
# Create a StockIndexDataset object for the LSTM network
LSTMdata = StockIndexDataset(datasets_path[0], time_steps=LSTM_params['time_steps'],
                             forecast_steps=LSTM_params['forecast_steps'],
                             batch_size=LSTM_params['batch_size'], val_ratio=.1, test_ratio=.1)
LSTMdata.info()

In [ ]:
# Build the Simple RNN network
RNN_input = (RNNdata.batch_size, RNNdata.time_steps, RNNdata.X_train.shape[2])
RNN_model = build_SimpleRNN(RNN_input, RNN_params)

# Define callbacks for later model training
# Add 'mcp' keyword into the "callbacks" list for 
# saving the best weights during model traning

RNN_filepath = "RNN_bestweights.hdf5"  
# change to your own working directory for a specific model
# output_dir = os.path.dirname(OUTPUT_PATH)
RNN_callbacks = training_callbacks(['reduce_lr', 'mcp'], params = RNN_params,
                                   filepath = RNN_filepath)

In [ ]:
# Build the Rnn network with GRU units
GRU_input = (GRUdata.batch_size, GRUdata.time_steps, GRUdata.X_train.shape[2])
GRU_model = build_GRU(GRU_input, GRU_params)

GRU_filepath = "GRU_bestweights.hdf5"  
# change to your own working directory for a specific model
GRU_callbacks = training_callbacks(['reduce_lr', 'mcp'], params = GRU_params,
                                   filepath = GRU_filepath)

In [ ]:
# Build the Rnn network with LSTM units
LSTM_input = (LSTMdata.batch_size, LSTMdata.time_steps, LSTMdata.X_train.shape[2])
LSTM_model = build_LSTM(LSTM_input, LSTM_params)

LTSM_filepath = "LSTM_bestweights.hdf5"  
# change to your own working directory for a specific model
LSTM_callbacks = training_callbacks(['reduce_lr', 'mcp'], params = LSTM_params,
                               filepath = LTSM_filepath)

In [ ]:
%%time
# Train Simple RNN model
RNN_hist = RNN_model.fit(RNNdata.X_train, RNNdata.y_train, 
                         epochs=RNN_params["epochs"], batch_size=RNNdata.batch_size,
                         shuffle=False, validation_data=(RNNdata.X_val, RNNdata.y_val), 
                         verbose=2, callbacks=RNN_callbacks)

In [ ]:
%%time
# Train Simple GRU model
GRU_hist = GRU_model.fit(GRUdata.X_train, GRUdata.y_train, 
                         epochs=GRU_params["epochs"], batch_size=GRUdata.batch_size,
                         shuffle=False, validation_data=(GRUdata.X_val, GRUdata.y_val), 
                         verbose=2, callbacks=GRU_callbacks)

In [ ]:
%%time
# Train LSTM model
LSTM_hist = LSTM_model.fit(LSTMdata.X_train, LSTMdata.y_train, 
                          epochs=LSTM_params["epochs"], batch_size=LSTMdata.batch_size,
                          shuffle=False, validation_data=(LSTMdata.X_val, LSTMdata.y_val), 
                          verbose=2, callbacks=LSTM_callbacks)

In [ ]:
# Create a ModelPredictions object containing results from the Simple RNN model 
RNN_predictions = ModelPredictions(RNN_model, 
                                   X = RNNdata.X, 
                                   y = RNNdata.y, 
                                   raw_y = RNNdata.raw_seq, 
                                   datetime = RNNdata.datetime,
                                   window = RNN_params['window'],
                                   batch_size = RNN_params['batch_size'])

In [ ]:
# Plot the results on training, validation and test data sets for Simple RNN model
labels = ['Actual', 'Training', 'Validation', 'Test']
RNN_predictions.plot_predictions(title="SH300IF (RNN model)", labels=labels)

In [ ]:
# Create a ModelPredictions object containing results from the GRU model 
GRU_predictions = ModelPredictions(GRU_model, 
                                   X = GRUdata.X, 
                                   y = GRUdata.y, 
                                   raw_y = GRUdata.raw_seq, 
                                   datetime = GRUdata.datetime,
                                   window = GRU_params['window'],
                                   batch_size = GRU_params['batch_size'])

In [ ]:
# Plot the results on training, validation and test data sets for RNN-GRU model
labels = ['Actual', 'Training', 'Validation', 'Test']
GRU_predictions.plot_predictions(title="SH300IF (GRU model)", labels=labels)

In [ ]:
# Create a ModelPredictions object containing results from the LSTM model 
LSTM_predictions = ModelPredictions(LSTM_model, 
                                    X = LSTMdata.X, 
                                    y = LSTMdata.y, 
                                    raw_y = LSTMdata.raw_seq, 
                                    datetime = LSTMdata.datetime,
                                    window = LSTM_params['window'],
                                    batch_size = LSTM_params['batch_size'])

In [ ]:
# Plot the results on training, validation and test data sets for RNN-LSTM model
labels = ['Actual', 'Training', 'Validation', 'Test']
LSTM_predictions.plot_predictions(title="SH300IF (LSTM model)", labels=labels)

In [ ]:
# Plot and compare the results obtained from the three RNNs on the test data set
testset = [RNN_predictions.true_org[2], 
           RNN_predictions.predictions_org[2],
           GRU_predictions.predictions_org[2],
           LSTM_predictions.predictions_org[2]]
datetime = pd.Series(RNNdata.datetime[2].values, 
                     index = RNN_predictions.true_org[2].index)
labels = ['Actual', 'RNN', 'GRU', 'LSTM']
list_mse = [RNN_predictions.loss[2],
       GRU_predictions.loss[2],
       LSTM_predictions.loss[2]]
list_mda = [RNN_predictions.mda[2],
       GRU_predictions.mda[2],
       LSTM_predictions.mda[2]]
labels_loss = create_labels(labels, list_mse, list_mda)
plot_seqs(testset, datetime=datetime, labels=labels_loss)

In [ ]:
# Load previously saved models, including architecture, weights and optimizer states
dependencies = {'rmse': rmse,
                'mda': mda}
saved_RNN = load_model(RNN_filepath, custom_objects=dependencies)

In [ ]:
# Create a StockIndexDataset object that contains new data for testing the SimpleRNN network
# Create the object with the key "test_only=True"

test_data = StockIndexDataset(datasets_path[-1], time_steps=RNN_params['time_steps'], 
                              forecast_steps=RNN_params['forecast_steps'],
                              batch_size=RNN_params['batch_size'], test_only=True)
test_data.info()

In [ ]:
# Create a ModelPredictions object containing results of the Simple RNN model on the test data set 
RNN_test = ModelPredictions(saved_RNN, 
                            X = test_data.X, 
                            y = test_data.y, 
                            raw_y = test_data.raw_seq, 
                            datetime = test_data.datetime,
                            window = RNN_params['window'],
                            batch_size = RNN_params['batch_size'])

In [ ]:
# Plot the testing results
labels = ['Actual', 'Test']
RNN_test.plot_predictions(title="SH300IF (RNN model)", labels=labels)